In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import logging 
import os

# Path to the log file
log_file_path = 'spam.log'

# Remove the log file if it exists
if os.path.exists(log_file_path):
    os.remove(log_file_path)

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
# create file handler which logs even debug messages
fh = logging.FileHandler(log_file_path, mode='w')
# Create a formatter and set it for the handler
formatter = logging.Formatter('%(name)s|%(levelname)s| %(message)s')
fh.setFormatter(formatter)
fh.setLevel(logging.DEBUG)
logger.addHandler(fh)

import json
import polars as pl
from polars import col, lit
import altair as alt
import simrs
from history import History, tabular
import input_builder as b
import utils 

In [3]:
def small_scenario():
  # Agents
  agent_ids = ["a" +str(i) for i in range(1, 10)] 

  # Goods
  wheat = "Wheat"
  goods = [wheat]

  # Ports
  genoa = "Genoa"
  rome = "Rome"
  milan = "Milan"
  venice = "Venice"
  marsailles = "Marsailles"
  port_ids = [genoa, rome, milan, venice, marsailles]
  # port_ids = [genoa, rome]

  # genoa -> milan -> marsailles
  #   v         v
  # rome -> venice
  edges = [(genoa, milan), (milan, marsailles), (rome, venice), (rome, genoa), (venice, milan)]
  # edges = [(genoa, rome)]
  # _market = lambda net: b._market(wheat, b._market_info(net=net)) 
  _market = lambda x: b._market(wheat, b._market_info(production=10, consumption=10-x, supply=1000+x)) 

  # net balanced
  ports = [
    b._port(genoa, _market(2)), 
    b._port(milan, _market(1)), 
    b._port(rome, _market(0)), 
    b._port(venice, _market(-1)),
    b._port(marsailles, _market(-2)), 
  ]

  _agent = lambda id, pos: b._agent(id, pos, 1000, "Exhaustive")
  agents = [_agent(id, port_id) for (id, port_id) in zip(agent_ids, port_ids * 20)] 

  x = b._inputFormat( agents = agents, ports=ports, edges= edges, opts=b._opts(ticks=50))
  with open("../../input/last.json", 'w') as fp:
    json.dump(x, fp, indent=2)
  return x

def barbell_scenario():
  # Agents
  agent_ids = ["a" +str(i) for i in range(1, 10)] 

  # Goods
  wheat = "Wheat"
  goods = [wheat]

  # Ports
  la = "la"
  lb = "lb"
  m1 = "m1"
  m2 = "m2"
  m3 = "m3"
  ra = "ra"
  # rb = "rb"
  port_ids = [la, lb, m1, m2, m3, ra]

  
  edges = [(la, lb), (la, m1), (lb, m1), (m1, m2), (m2, m3), (m3, ra), ]
  # _market = lambda net: b._market(wheat, b._market_info(net=net)) 
  _market = lambda c,s: b._market(wheat, b._market_info(production=100, consumption=100+c, supply=1000-s, pricer=b._inversep(100_000))) 

  # net balance
  ports = [
    b._port(la, _market(0, 20)),
    b._port(lb, _market(0, 20)),
    b._port(m1, _market(0, 20)),
    b._port(m2, _market(0, 0)),
    b._port(m3, _market(0, 0)),
    b._port(ra, _market(0, -20)),
    # b._port(rb, _market(0, -20)),
  ]
  # la
  #     m1 m2 m3 ra
  # lb

  _agent = lambda id, pos: b._agent(id, pos, 1000, "Exhaustive")
  agents = [_agent(id, port_id) for (id, port_id) in zip(agent_ids, [la] * 20)] 

  x = b._inputFormat( agents = agents, ports=ports, edges= edges, opts=b._opts(ticks=100))
  with open("../../input/last.json", 'w') as fp:
    json.dump(x, fp, indent=2)
  return x

history = simrs.run(barbell_scenario())
(actions, agents, markets, events) = tabular(history)
markets = markets.select(pl.exclude("pricer"))

In [4]:
print("agents") 
agents

agents


behavior,cargo,coins,id,pos,tick
str,str,f64,str,str,i64
"""Exhaustive""",null,1000.0,"""a9""","""la""",0
"""Exhaustive""",null,1000.0,"""a1""","""la""",0
"""Exhaustive""",null,1000.0,"""a5""","""la""",0
"""Exhaustive""",null,1000.0,"""a8""","""la""",0
"""Exhaustive""",null,1000.0,"""a6""","""la""",0
"""Exhaustive""",null,1000.0,"""a2""","""la""",0
"""Exhaustive""",null,1000.0,"""a7""","""la""",0
"""Exhaustive""",null,1000.0,"""a3""","""la""",0
"""Exhaustive""",null,1000.0,"""a4""","""la""",0


In [5]:
print("markets") 
markets

markets


consumption,good,port,price,production,supply,tick
f64,str,str,f64,f64,f64,i64
100.0,"""Wheat""","""m2""",100.0,100.0,1000.0,0
100.0,"""Wheat""","""la""",102.040816,100.0,980.0,0
100.0,"""Wheat""","""lb""",102.040816,100.0,980.0,0
100.0,"""Wheat""","""ra""",98.039216,100.0,1020.0,0
100.0,"""Wheat""","""m3""",100.0,100.0,1000.0,0
100.0,"""Wheat""","""m1""",102.040816,100.0,980.0,0
100.0,"""Wheat""","""m3""",100.0,100.0,1000.0,1
100.0,"""Wheat""","""lb""",102.040816,100.0,980.0,1
100.0,"""Wheat""","""ra""",98.039216,100.0,1020.0,1


In [6]:
print("actions")
actions.head(10)

actions


action,agent_id,port_id,tick,good
str,str,str,i64,str
"""Move""","""a9""","""m1""",0,null
"""Move""","""a1""","""m1""",0,null
"""Move""","""a5""","""m1""",0,null
"""Move""","""a8""","""m1""",0,null
"""Move""","""a6""","""m1""",0,null
"""Move""","""a2""","""m1""",0,null
"""Move""","""a7""","""m1""",0,null
"""Move""","""a3""","""m1""",0,null
"""Move""","""a4""","""m1""",0,null


In [7]:
events.head(10)

agent,amt,cost,event,good,port,tick
str,i64,f64,str,str,str,i64
"""a9""",1,100.05005,"""Trade""","""Wheat""","""m2""",2
"""a1""",1,100.15025,"""Trade""","""Wheat""","""m2""",2
"""a5""",1,100.250652,"""Trade""","""Wheat""","""m2""",2
"""a8""",1,100.351255,"""Trade""","""Wheat""","""m2""",2
"""a6""",1,100.452059,"""Trade""","""Wheat""","""m2""",2
"""a2""",1,100.553067,"""Trade""","""Wheat""","""m2""",2
"""a7""",1,100.654278,"""Trade""","""Wheat""","""m2""",2
"""a3""",1,100.755693,"""Trade""","""Wheat""","""m2""",2
"""a4""",1,100.857313,"""Trade""","""Wheat""","""m2""",2


In [8]:
def plot_agents(agents: pl.DataFrame):
    base = alt.Chart(agents).encode(
        x='tick:Q',
        y=alt.Y('coins:Q').scale(zero=False),
        color=alt.Color('id:O').scale(scheme='dark2'),
    )
    lines = base.transform_loess('tick', 'coins', bandwidth=.5, groupby=['id']).mark_line(size=4)
    return (base.mark_point() + lines).interactive() 
  
plot_agents(agents)

alt.LayerChart(...)

In [27]:
def plot_agent_locations(agents: pl.DataFrame):
    base = alt.Chart(agents).encode(
        x='tick:Q',
        y='pos:N',
        color=alt.Color('id:O').scale(scheme='dark2'),
    )
    return base.mark_point().interactive()
plot_agent_locations(agents)

alt.Chart(...)

In [42]:
def plot_prices_by_port(ports: pl.DataFrame, color='dark2'):
    base = alt.Chart(ports).encode(
        x='tick:Q',
        y=alt.Y('price:Q', scale=alt.Scale(zero=False)),
        color=alt.Color('port:O').scale(scheme=color),
    )
    lines = base.transform_loess('tick', 'price', bandwidth=.1, groupby=['port']).mark_line(size=4)
    return (base.mark_point() + lines).interactive() 
    # return lines.interactive()
plot_prices_by_port(markets)

alt.LayerChart(...)

In [43]:
## Where would prices have been if agents didn't trade?
# x = b._inputFormat( agents = agents, ports=ports, edges= edges, opts=b._opts(ticks=100))
def no_agent_markets(input_format) -> pl.DataFrame:
    input_format['agents'] = []
    no_agent_history = simrs.run(input_format)
    (_, _, no_agent_markets,_) = tabular(no_agent_history)
    return no_agent_markets.select(pl.exclude("pricer"))

plot_prices_by_port(no_agent_markets(barbell_scenario())) 

alt.LayerChart(...)

In [29]:
def make_routes(events):
    trade_events = events.filter(events["event"] == "Trade")
    def foo(df):
        buys = df.filter(df["amt"] > 0).select(
            'agent', 
            'amt', 
            pl.col("cost").alias('buy_cost'), 
            pl.col('port').alias('src'), 
            pl.col('tick').alias('buy_tick')
        )
        sells = df.filter(df["amt"] < 0).select(
            pl.col('cost').alias('sell_cost'), 
            pl.col('port').alias('dst'),
            pl.col('tick').alias('sell_tick')
        )

        df = pl.concat( [ buys, sells ], how='horizontal')
        df = df.with_columns((-df["sell_cost"] - df["buy_cost"]).alias("profit"))
        df = df.with_columns((df['buy_cost'] / df['amt']).alias('buy_price'))
        df = df.with_columns((df['sell_cost'] / -df['amt']).alias('sell_price'))
        return df

    return trade_events.groupby('agent').apply(foo)
routes = make_routes(events)
blag = routes.with_columns(pl.struct("src", "dst").alias("route"))
routes

agent,amt,buy_cost,src,buy_tick,sell_cost,dst,sell_tick,profit,buy_price,sell_price
str,i64,f64,str,i64,f64,str,i64,f64,f64,f64
"""a1""",1,100.15025,"""m2""",2,-99.85025,"""m3""",3,-0.300001,100.15025,99.85025
"""a1""",1,99.255608,"""m3""",4,-100.755693,"""m2""",5,1.500086,99.255608,100.755693
"""a1""",1,100.05005,"""m2""",6,-99.95005,"""m3""",7,-0.1,100.05005,99.95005
"""a1""",1,99.354222,"""m3""",8,-100.654278,"""m2""",9,1.300056,99.354222,100.654278
"""a1""",1,100.05005,"""m2""",10,-99.95005,"""m3""",11,-0.1,100.05005,99.95005
"""a1""",1,99.453033,"""m3""",12,-100.654278,"""m2""",13,1.201245,99.453033,100.654278
"""a1""",1,100.15025,"""m2""",14,-100.05005,"""m3""",15,-0.1002,100.15025,100.05005
"""a1""",1,98.280122,"""ra""",17,-99.85025,"""m3""",18,1.570128,98.280122,99.85025
"""a1""",1,99.85025,"""m3""",19,-100.05005,"""m2""",20,0.1998,99.85025,100.05005


In [30]:
blag.groupby("route").agg(pl.count()).sort("count", descending=True)


route,count
struct[2],u32
"{""m3"",""m2""}",88
"{""m2"",""m3""}",73
"{""ra"",""m3""}",49
"{""m2"",""m1""}",38
"{""m3"",""ra""}",33
"{""m1"",""lb""}",21
"{""m1"",""m2""}",16
"{""m3"",""m3""}",16
"{""lb"",""la""}",13


In [31]:
def plot_trades(trades: pl.DataFrame):
    base = alt.Chart(trades).encode(
        x='sell_tick:Q',
        y='profit:Q',
        color=alt.Color('agent:N').scale(scheme='dark2'),
    )
    return base.mark_point().interactive()
plot_trades(routes)

alt.Chart(...)

In [32]:
def plot_buy_and_sell_prices(trades: pl.DataFrame):
    buy = alt.Chart(trades).encode(
        x='buy_tick:Q',
        y='buy_price:Q',
        color=alt.Color('dst:N').scale(scheme='dark2'),
    )
    sell = alt.Chart(trades).encode(
        x='sell_tick:Q',
        y='sell_price:Q',
        color=alt.Color('dst:N').scale(scheme='dark2'),
    )
    
    return (buy.mark_point()+sell.mark_point()).interactive()

plot_buy_and_sell_prices(routes) + plot_trades(routes)

alt.LayerChart(...)

alt.LayerChart(...)

In [38]:
plot_prices_by_port(markets)

alt.LayerChart(...)

## Port level analysis
- How much does each port trade?
- Trade volume bucketted 
- Volume in dollars 
- Biggest trading partners (other ports)

In [35]:
## Total goods traded per port
events.groupby("port").agg(pl.sum("amt"))
# def plot_

port,amt
str,i64
"""m3""",4
"""ra""",20
"""la""",-7
"""lb""",-8
"""m1""",-9
"""m2""",5


Metrics for individual agent
- total coins
- coins per tick
- died?

Metrics for population of agents
- min, max, median, mean, std of agent coins

How well did the agents equalize prices?
- box plot of prices
- stddev of pricesf

Construct 'trades'
- bought Cargo at StartPort for BuyPrice
- sold Cargo at EndPort for SellPrice
- profit = SellPrice - BuyPrice
- Route = StartPort -> .. ->  EndPort
- RouteLength = len(Route)